In [ ]:
#customer_data table

In [2]:
import csv
import uuid
from faker import Faker
from datetime import datetime, timedelta

def generate_fake_customer_data(num_customers):
    fake = Faker('en_IN')
    data = []

    # Generate data for each customer
    for _ in range(num_customers):
        customer_id = uuid.uuid4()
        first_name = fake.first_name()
        last_name = fake.last_name()
        email = f"{first_name.lower()}.{last_name.lower()}@gmail.com"
        
        # Generate signup dates for the last 5 years
        signup_date = fake.date_between(start_date='-5y', end_date='today')
        
        data.append({
            'CustomerID': str(customer_id),
            'FirstName': first_name,
            'LastName': last_name,
            'Email': email,
            'SignUpDate': signup_date.strftime('%Y-%m-%d')
        })

    return data

def save_data_to_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['CustomerID', 'FirstName', 'LastName', 'Email', 'SignUpDate']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

if __name__ == "__main__":
    num_customers = 15000
    customer_data = generate_fake_customer_data(num_customers)
    save_data_to_csv(customer_data, 'customer_data.csv')
    print("Data generated and saved to customer_data.csv")


Data generated and saved to customer_data.csv


In [ ]:
#order_data table

In [25]:
import pandas as pd
import numpy as np
import uuid
import random
from faker import Faker
from datetime import datetime, timedelta

# Load customer data from the CSV file
customer_data = pd.read_csv('/home/nineleaps/Downloads/customer_data.csv')

# Define the price ranges for products (in INR)
# Define price ranges for categories and product groupsfactor
# Define the mapping of sub-categories to their respective categories
sub_category_to_category = {
    "Mobile Phones": "Electronics",
    "Laptops": "Electronics",
    "Televisions": "Electronics",
    "Cameras": "Electronics",
    "Headphones & Earphones": "Electronics",
    "Dresses": "Clothing",
    "Shirts & Tops": "Clothing",
    "Cookware": "Home & Kitchen",
    "Home Decor": "Home & Kitchen",
    "Bedding & Linens": "Home & Kitchen",
    "Fiction Books": "Books",
    "Non-Fiction Books": "Books",
    "Sports Equipment": "Sports & Outdoors",
    "Makeup & Skincare": "Beauty & Personal Care",
    "Haircare": "Beauty & Personal Care",
    "Board Games": "Toys & Games",
    "Baby & Toddler Toys": "Toys & Games",
    "Vitamins & Supplements": "Health & Wellness",
    "Fitness Equipment": "Health & Wellness",
    "Car Accessories": "Automotive",
    "Office Stationery": "Office Supplies"
}

# Function to generate random orders
def generate_orders(num_orders, start_date, end_date):
    fake = Faker('en_IN')
    orders = []

    for _ in range(num_orders):
        # Generate random order details
        order_id = str(uuid.uuid4())
        customer = customer_data.sample()
        CustomerID = customer['CustomerID'].values[0]
        signup_date = pd.to_datetime(customer['SignUpDate'].values[0])
        order_date = start_date + pd.DateOffset(days=random.randint(0, (end_date - start_date).days))

        # Remove time from the timestamp
        order_date = order_date.replace(hour=0, minute=0, second=0, microsecond=0)

        # Calculate seasonal factor based on special occasions
        seasonal_factor = 1.0
        for occasion, (occasion_start, occasion_end, factor) in occasions.items():
            occasion_start_date = datetime.strptime(f"{order_date.year}-{occasion_start}", "%Y-%m-%d")
            occasion_end_date = datetime.strptime(f"{order_date.year}-{occasion_end}", "%Y-%m-%d")
            if is_date_in_range(order_date, occasion_start_date, occasion_end_date):
                seasonal_factor = factor
                break

        # Choose a random category
        category = random.choice(list(category_price_ranges.keys()))

        # Get the sub-category based on the chosen category
        sub_categories = [sub_category for sub_category, cat in sub_category_to_category.items() if cat == category]
        sub_category = random.choice(sub_categories)

        # Generate random price within the specified range and round off
        category_price_range = category_price_ranges[category]
        product_group_price_range = product_group_price_ranges[sub_category]
        price = np.mean([np.random.uniform(*category_price_range), np.random.uniform(*product_group_price_range)])
        price = round(price)

        # Apply the seasonal factor to the price
        price *= seasonal_factor

        # Append the order details to the orders list
        orders.append({
            "order_id": order_id,
            "CustomerID": CustomerID,
            "order_date": order_date,
            "total_amount": price,
            "category": category,
            "sub_category": sub_category
        })

    return orders

# ... (rest of the code remains unchanged)


if __name__ == "__main__":
    # Define the date range for the last 5 years
    end_date = datetime.now()
    start_date = end_date - timedelta(days=5 * 365)

    # Generate order data
    num_orders = random.randint(50000,50001) # Randomly choose the number of orders in the specified range
#     num_orders = random.randint(164250, 172750)  # Randomly choose the number of orders in the specified range
    order_data = generate_orders(num_orders, start_date, end_date)

    # Save the order data to a CSV file
    df_orders = pd.DataFrame(order_data)
    df_orders.to_csv('orders_data.csv', index=False)

    print("Order data generated and saved to orders_data.csv.")


Order data generated and saved to orders_data.csv.


In [ ]:
#product_performance_data

In [30]:
import pandas as pd
import numpy as np
import uuid
import random
from faker import Faker

# Load data from the previously created CSV files
orders_data = pd.read_csv('/home/nineleaps/Downloads/orders_data.csv')
customer_data = pd.read_csv('/home/nineleaps/Downloads/customer_data.csv')

# Define the mapping of sub-categories to their respective categories
sub_category_to_category = {
    "Mobile Phones": "Electronics",
    "Laptops": "Electronics",
    "Televisions": "Electronics",
    "Cameras": "Electronics",
    "Headphones & Earphones": "Electronics",
    "Dresses": "Clothing",
    "Shirts & Tops": "Clothing",
    "Cookware": "Home & Kitchen",
    "Home Decor": "Home & Kitchen",
    "Bedding & Linens": "Home & Kitchen",
    "Fiction Books": "Books",
    "Non-Fiction Books": "Books",
    "Sports Equipment": "Sports & Outdoors",
    "Makeup & Skincare": "Beauty & Personal Care",
    "Haircare": "Beauty & Personal Care",
    "Board Games": "Toys & Games",
    "Baby & Toddler Toys": "Toys & Games",
    "Vitamins & Supplements": "Health & Wellness",
    "Fitness Equipment": "Health & Wellness",
    "Car Accessories": "Automotive",
    "Office Stationery": "Office Supplies"
}

# Define the price ranges for products (in INR)
category_price_ranges = {
    "Electronics": (5000, 100000),  # Price range for Electronics category (in INR)
    "Clothing": (500, 10000),      # Price range for Clothing category (in INR)
    "Home & Kitchen": (100, 5000), # Price range for Home & Kitchen category (in INR)
    "Books": (50, 2000),           # Price range for Books category (in INR)
    "Sports & Outdoors": (100, 5000),    # Price range for Sports & Outdoors category (in INR)
    "Beauty & Personal Care": (100, 3000),   # Price range for Beauty & Personal Care category (in INR)
    "Toys & Games": (50, 2000),    # Price range for Toys & Games category (in INR)
    "Health & Wellness": (100, 5000),    # Price range for Health & Wellness category (in INR)
    "Automotive": (500, 10000),    # Price range for Automotive category (in INR)
    "Office Supplies": (50, 1000)   # Price range for Office Supplies category (in INR)
}

# Define the price ranges for product groups (in INR)
product_group_price_ranges = {
    "Smartphones": (10000, 80000),    # Price range for Smartphones (in INR)
    "Laptops": (30000, 150000),       # Price range for Laptops (in INR)
    "Televisions": (20000, 100000),   # Price range for Televisions (in INR)
    "Cameras": (15000, 80000),        # Price range for Cameras (in INR)
    "Headphones & Earphones": (500, 5000),    # Price range for Headphones & Earphones (in INR)
    "Dresses": (500, 5000),           # Price range for Dresses (in INR)
    "Shirts & Tops": (300, 3000),     # Price range for Shirts & Tops (in INR)
    "Cookware": (300, 5000),          # Price range for Cookware (in INR)
    "Home Decor": (200, 3000),        # Price range for Home Decor (in INR)
    "Bedding & Linens": (500, 3000),  # Price range for Bedding & Linens (in INR)
    "Fiction Books": (100, 1000),     # Price range for Fiction Books (in INR)
    "Non-Fiction Books": (100, 2000), # Price range for Non-Fiction Books (in INR)
    "Sports Equipment": (100, 5000),  # Price range for Sports Equipment (in INR)
    "Makeup & Skincare": (100, 2000), # Price range for Makeup & Skincare (in INR)
    "Haircare": (100, 1500),          # Price range for Haircare (in INR)
    "Board Games": (200, 2000),       # Price range for Board Games (in INR)
    "Baby & Toddler Toys": (100, 2000),   # Price range for Baby & Toddler Toys (in INR)
    "Vitamins & Supplements": (200, 3000),    # Price range for Vitamins & Supplements (in INR)
    "Fitness Equipment": (500, 10000),   # Price range for Fitness Equipment (in INR)
    "Car Accessories": (100, 5000),      # Price range for Car Accessories (in INR)
    "Office Stationery": (50, 500),       # Price range for Office Stationery (in INR)
}

# Function to generate random product performance data with relevant product names
def generate_product_performance(num_products):
    fake = Faker('en_IN')
    product_performance = []

    for _ in range(num_products):
        # Generate random product details
        product_id = str(uuid.uuid4())
        category = random.choice(list(category_price_ranges.keys()))

        # Get the sub-category based on the chosen category
        sub_categories = [sub_category for sub_category, cat in sub_category_to_category.items() if cat == category]
        sub_category = random.choice(sub_categories)

        # Generate a relevant product name based on the sub-category
        relevant_product_names = {
            "Mobile Phones": ["Smartphone", "Cell Phone", "Android Phone", "iPhone", "Mobile Device"],
            "Laptops": ["Notebook", "UltraBook", "Gaming Laptop", "Business Laptop"],
            "Televisions": ["Smart TV", "LED TV", "4K TV", "Curved TV"],
            "Cameras": ["DSLR Camera", "Mirrorless Camera", "Point-and-Shoot Camera"],
            "Headphones & Earphones": ["Wireless Headphones", "Over-Ear Headphones", "In-Ear Earphones"],
            "Dresses": ["Maxi Dress", "Shift Dress", "Sundress", "Evening Gown"],
            "Shirts & Tops": ["T-shirt", "Polo Shirt", "Button-Up Shirt", "Blouse"],
            "Cookware": ["Non-Stick Cookware", "Stainless Steel Cookware", "Cast Iron Cookware"],
            "Home Decor": ["Wall Art", "Decorative Pillows", "Curtains", "Rugs"],
            "Bedding & Linens": ["Bed Sheets", "Comforter Set", "Duvet Cover", "Pillow Cases"],
            "Fiction Books": ["Mystery Novel", "Romance Novel", "Science Fiction Book"],
            "Non-Fiction Books": ["Self-Help Book", "Cookbook", "Biography", "History Book"],
            "Sports Equipment": ["Yoga Mat", "Dumbbells", "Basketball", "Tennis Racket"],
            "Makeup & Skincare": ["Lipstick", "Foundation", "Face Mask", "Eye Cream"],
            "Haircare": ["Shampoo", "Conditioner", "Hair Oil", "Hair Styling Gel"],
            "Board Games": ["Strategy Board Game", "Party Game", "Card Game"],
            "Baby & Toddler Toys": ["Stacking Toys", "Activity Center", "Stuffed Animal"],
            "Vitamins & Supplements": ["Multivitamins", "Fish Oil Supplement", "Protein Powder"],
            "Fitness Equipment": ["Treadmill", "Exercise Bike", "Resistance Bands"],
            "Car Accessories": ["Car Phone Mount", "Seat Covers", "Car Air Freshener"],
            "Office Stationery": ["Notebook", "Pen Set", "Desk Organizer"]
            # Add relevant product names for other sub-categories as needed
        }
        product_name = random.choice(relevant_product_names.get(sub_category, ["Product"])) + " " + sub_category

        # Generate random number of units sold and revenue generated
        units_sold = random.randint(100, 1000)
        price = np.mean([np.random.uniform(*category_price_ranges[category]), np.random.uniform(*product_group_price_ranges.get(sub_category, (100, 2000)))])
        revenue_generated = round(price * units_sold)

        # Append the product performance details to the list
        product_performance.append({
            "ProductID": product_id,
            "ProductName": product_name,
            "Category": category,
            "UnitsSold": units_sold,
            "RevenueGenerated": revenue_generated
        })

    return product_performance

if __name__ == "__main__":
    # Define the number of products to generate
    num_products = 25000

    # Generate product performance data
    product_performance_data = generate_product_performance(num_products)

    # Save the product performance data to a CSV file
    df_product_performance = pd.DataFrame(product_performance_data)
    df_product_performance.to_csv('product_performance_data.csv', index=False)

    print("Product Performance data generated and saved to product_performance_data.csv.")


Product Performance data generated and saved to product_performance_data.csv.


In [ ]:
#shopping_cart_data table

In [37]:
import pandas as pd
import uuid
import random
from faker import Faker
from datetime import datetime, timedelta

# Load existing Customers and ProductPerformance data
customers_data = pd.read_csv('/home/nineleaps/Downloads/customer_data.csv')
product_performance_data = pd.read_csv('/home/nineleaps/Downloads/product_performance_data.csv')

# Get the current date
current_date = datetime.now()

# Function to generate ShoppingCarts data
def generate_shopping_carts(customers_data, product_performance_data, num_carts_per_customer):
    fake = Faker('en_IN')
    shopping_carts = []

    for _, customer in customers_data.iterrows():
        CustomerID = customer['CustomerID']
        SignUpDate = datetime.strptime(customer['SignUpDate'], '%Y-%m-%d')

        for _ in range(num_carts_per_customer):
            cart_id = str(uuid.uuid4())
            days_after_signup = random.randint(1, 365 * 5)  # Generate days within the last 5 years
            date_added_to_cart = SignUpDate + timedelta(days=days_after_signup)

            # Make sure the date is not in the future or after the current date
            if date_added_to_cart > current_date:
                date_added_to_cart = current_date

            product = product_performance_data.sample(n=1)
            product_id = product['ProductID'].values[0]

            shopping_carts.append({
                'CartID': cart_id,
                'CustomerID': CustomerID,
                'DateAddedToCart': date_added_to_cart.strftime('%Y-%m-%d'),  # Convert date to string
                'ProductID': product_id
            })

    return shopping_carts

# Define the number of shopping carts per customer
num_carts_per_customer = 5

# Generate ShoppingCarts data
shopping_carts_data = generate_shopping_carts(customers_data, product_performance_data, num_carts_per_customer)

# Convert the data to a DataFrame
df_shopping_carts = pd.DataFrame(shopping_carts_data)

# Save the ShoppingCarts data to a CSV file
df_shopping_carts.to_csv('shopping_carts_data.csv', index=False)

print("ShoppingCarts data generated and saved to shopping_carts_data.csv.")


ShoppingCarts data generated and saved to shopping_carts_data.csv.


In [ ]:
#conversion_data table

In [38]:
import pandas as pd
import uuid
import random
from faker import Faker
from datetime import datetime, timedelta

# Get the current date
current_date = datetime.now()

# Function to generate Conversion Data (Visitors)
def generate_conversion_data(customers_data, product_performance_data, num_visitors):
    fake = Faker('en_IN')
    conversion_data = []

    for _ in range(num_visitors):
        visitor_id = str(uuid.uuid4())
        page_id = str(uuid.uuid4())
        date_visited = fake.date_between(start_date='-5y', end_date='today')

        # Check for holiday seasons, special occasions, and major shopping events
        holiday_seasons = ['Christmas', 'Thanksgiving', 'Black Friday']
        special_occasions = ["Valentine's Day", "Mother's Day"]
        major_shopping_events = ['Amazon Prime Day']

        action = 'purchase' if random.random() < 0.3 else 'not_purchase'  # 30% conversion rate

        # Increase conversion rate during holiday seasons
        if date_visited.month == 12 and date_visited.day >= 20:
            if random.random() < 0.8:
                action = 'purchase'
        elif date_visited.month == 11 and date_visited.day >= 20:
            if random.random() < 0.6:
                action = 'purchase'

        # Increase conversion rate during special occasions
        if date_visited.strftime("%B %d") in special_occasions:
            if random.random() < 0.7:
                action = 'purchase'

        # Increase conversion rate during major shopping events
        if date_visited.strftime("%B %d") in major_shopping_events:
            if random.random() < 0.8:
                action = 'purchase'

        conversion_data.append({
            'VisitorID': visitor_id,
            'PageID': page_id,
            'DateVisited': date_visited.strftime('%Y-%m-%d'),  # Convert date to string
            'Action': action
        })

    return conversion_data

# Define the number of visitors
num_visitors = 15000

# Generate Conversion Data (Visitors)
conversion_data = generate_conversion_data(customers_data, product_performance_data, num_visitors)

# Convert the data to a DataFrame
df_conversion_data = pd.DataFrame(conversion_data)

# Save the Conversion Data (Visitors) to a CSV file
df_conversion_data.to_csv('conversion_data.csv', index=False)

print("Conversion Data (Visitors) generated and saved to conversion_data.csv.")


Conversion Data (Visitors) generated and saved to conversion_data.csv.


In [ ]:
#promotional_data table

In [ ]:
import csv
import random
import uuid
from faker import Faker
from datetime import date, timedelta
fake = Faker()
def generate_campaign_id():
    return str(uuid.uuid4())
def generate_campaign_name(start_date):
    if start_date.month == 6:
        return "season end sale"
    elif start_date.month == 12:
        return "christmas and new year"
    elif start_date.month == 2:
        return "valentines day"
    elif start_date.month == 7:
        return "prime day"
    elif start_date.month == 5:
        return "mother's day"
    elif start_date.month == 11 and start_date.day >= 15:
        return "thanks giving"
    else:
        return "normal day"
def generate_promotional_price(total_amount, campaign_name):
    if campaign_name == "season end sale":
        return round(random.uniform(0.7, 0.8) * total_amount, 2)
    elif campaign_name == "christmas and new year" or campaign_name == "prime day":
        return round(random.uniform(0.8, 0.9) * total_amount, 2)
    elif campaign_name == "valentines day":
        return round(random.uniform(0.8, 0.9) * total_amount, 2)
    elif campaign_name == "mother's day":
        return round(random.uniform(0.6, 0.7) * total_amount, 2)
    elif campaign_name == "thanks giving":
        return round(random.uniform(0.3, 0.4) * total_amount, 2)
    else:
        return round(random.uniform(0.05, 0.1) * total_amount, 2)
def generate_advertising_price(campaign_name):
    if campaign_name == "mother's day":
        return random.randint(5000, 15000)
    elif campaign_name == "season end sale" or campaign_name == "christmas and new year":
        return random.randint(20000, 100000)
    elif campaign_name == "prime day":
        return random.randint(50000, 100000)
    elif campaign_name == "thanks giving":
        return random.randint(5000, 20000)
    elif campaign_name == "valentines day":
        return random.randint(10000, 20000)
    else:
        return random.randint(5000, 10000)
def generate_promotional_data():
    promotional_data = []
    with open('product_performance_data.csv', 'r') as product_file:
        product_data = list(csv.reader(product_file))
        product_ids = [row[0] for row in product_data[1:]]  # Assuming product IDs are in the first column
    with open('orders_data.csv', 'r') as order_file:
        order_data = list(csv.reader(order_file))
        total_amount = [float(row[3]) for row in order_data[3:]]  # Assuming total_amount is in the second column
    current_year = date.today().year
    for _ in range(25000):  # Generating 25000 rows for each product ID
        product_id = random.choice(product_ids)
        start_date = fake.date_between_dates(date(current_year - 5, 1, 1), date(current_year, 12, 31))
        campaign_name = generate_campaign_name(start_date)
        # Now we take a random total_amount from the list of total_amounts
        total_amounts = random.choice(total_amount)
        promotional_price = generate_promotional_price(total_amounts, campaign_name)
        advertising_price = generate_advertising_price(campaign_name)
        if campaign_name == "season end sale":
            end_date = date(current_year, 7, 31)
        elif campaign_name == "christmas and new year":
            end_date = date(current_year, 12, 31)
        elif campaign_name == "valentines day":
            end_date = date(current_year, 2, 20)
        elif campaign_name == "prime day":
            end_date = date(current_year, 8, 1)
        elif campaign_name == "mother's day":
            end_date = date(current_year, 5, 15)
        elif campaign_name == "thanks giving":
            end_date = date(current_year, 11, 30)
        else:
            # For normal day, set the end_date as 1 day after the start_date
            end_date = start_date + timedelta(days=1)
        promotional_data.append([generate_campaign_id(), campaign_name, start_date.strftime('%Y-%m-%d'),
                                 end_date.strftime('%Y-%m-%d'), promotional_price, advertising_price, product_id])
    with open('promotional_data.csv', 'w', newline='') as promo_file:  # Changed the filename to 'promotional_data.csv'
        writer = csv.writer(promo_file)
        writer.writerow(['campaign_id', 'campaign_name', 'start_date', 'end_date', 'promotional_price',
                         'advertising_price', 'product_id'])
        writer.writerows(promotional_data)
if __name__ == "__main__":
    generate_promotional_data()

In [ ]:
#sales_data table

In [ ]:
import csv
import uuid
import random
import pandas as pd
def generate_uuid():
    return str(uuid.uuid4())
def generate_sales_data():
    # Read data from product.csv
    product_data = pd.read_csv('product_performance_data.csv')
    # Read data from promotional_data.csv
    promotional_data = pd.read_csv('promotional_data.csv')
    # Get unique product IDs and their corresponding categories
    product_ids = product_data['ProductID'].tolist()
    categories = product_data['Category'].tolist()
    # Get unique campaign names
    campaign_names = promotional_data['campaign_name'].unique().tolist()
    # Get Indian states as locations
    indian_states = [
        'Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar', 'Chhattisgarh',
        'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh', 'Jharkhand', 'Karnataka',
        'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya', 'Mizoram',
        'Nagaland', 'Odisha', 'Punjab', 'Rajasthan', 'Sikkim', 'Tamil Nadu', 'Telangana',
        'Tripura', 'Uttar Pradesh', 'Uttarakhand', 'West Bengal',
        'Chandigarh',  'Delhi', 'Puducherry'
    ]
    # Generate sales data
    sales_data = []
    for _ in range(35000):  # Generating 35000 rows
        sales_id = generate_uuid()
        ProductID = random.choice(product_ids)
        Category = categories[product_ids.index(ProductID)]
        campaign_name = random.choice(campaign_names)
        location = random.choice(indian_states)
        # Append the generated data to the sales_data list
        sales_data.append([sales_id, ProductID, campaign_name, Category, location])
    # Save sales_data to sales_data.csv
    with open('sales_data.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['SalesID', 'ProductID', 'CampaignName', 'Category', 'Location'])
        writer.writerows(sales_data)
if __name__ == "__main__":
    generate_sales_data()